In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
df = pd.read_json('../processed_data/pl_data_dt.json')
df.head()


,season,round,date,time,home_team,away_team,home_goals,away_goals,home_poss,away_poss,...,away_shots_on_goal_rolling_avg,away_conceded_shots_on_goal_rolling_avg,away_goalkeeper_saves_rolling_avg,away_blocked_shots_rolling_avg,away_shots_off_goal_rolling_avg,away_chances_rolling_avg,away_shot_creation_ratio_rolling_avg,away_target_ratio_rolling_avg,away_conversion_rate_rolling_avg,away_target_to_goal_ratio_rolling_avg
0,20232024,1,2023-11-08,15:00,Burnley,Manchester City,0,3,0.35,0.65,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20232024,1,2023-12-08,10:00,Sheffield Utd,Crystal Palace,0,1,0.33,0.67,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20232024,1,2023-12-08,10:00,Everton,Fulham,0,1,0.41,0.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20232024,1,2023-12-08,10:00,Brighton,Luton,4,1,0.71,0.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20232024,1,2023-12-08,10:00,Bournemouth,West Ham,1,1,0.63,0.37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load and prepare data
df = pd.read_json('../processed_data/pl_data_dt.json')

selected_features = [
    'home_shots_rolling_avg',
    'home_shots_on_goal_rolling_avg', 
    'home_target_to_goal_ratio_rolling_avg',
    'home_conversion_rate_rolling_avg',
    'home_shot_creation_ratio_rolling_avg', 
    'home_shots_off_goal_rolling_avg'
]

# Split data by date
cutoff_date = '2023-01-01'
df_past = df[df['date'] < cutoff_date]
df_future = df[df['date'] >= cutoff_date]

X_train = df_past[selected_features]
y_train = df_past['home_goals']
X_test = df_future[selected_features]
y_test = df_future['home_goals']

# Train model
model = RandomForestRegressor(max_depth=2, n_estimators=100, random_state=45)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE): %.4f" % mse)
print("Mean Absolute Error (MAE): %.4f" % mae)
print("R-squared (R2 Score): %.4f" % r2)

# Create results dataframe
results_df = pd.DataFrame({
    'Actual Goals': y_test.values,
    'Predicted Goals': y_pred
}, index=y_test.index)

if 'home_team' in df.columns:
    results_df['Home Team'] = df_future['home_team']

# Calculate training and test R² scores
train_r2 = r2_score(y_train, model.predict(X_train))
test_r2 = r2_score(y_test, y_pred)

print("\nTraining R² Score: %.4f" % train_r2)
print("Test R² Score: %.4f" % test_r2)

# Display sample predictions
print("\nSample Predictions:")
print(results_df.head(20))

Mean Squared Error (MSE): 1.6994
Mean Absolute Error (MAE): 1.0366
R-squared (R2 Score): 0.0708

Training R² Score: 0.1016
Test R² Score: 0.0708

Sample Predictions:
   Actual Goals  Predicted Goals      Home Team
0             0         1.222594        Burnley
1             0         1.222594  Sheffield Utd
2             0         1.222594        Everton
3             4         1.222594       Brighton
4             1         1.222594    Bournemouth


In [3]:
df.columns

Index(['season', 'round', 'date', 'time', 'home_team', 'away_team',
       'home_goals', 'away_goals', 'home_poss', 'away_poss', 'home_shots',
       'away_shots', 'home_shots_on_goal', 'away_shots_on_goal',
       'home_corner_kicks', 'away_corner_kicks', 'home_goalkeeper_saves',
       'away_goalkeeper_saves', 'home_attacks', 'away_attacks',
       'home_dangerous_attacks', 'away_dangerous_attacks',
       'home_blocked_shots', 'away_blocked_shots', 'home_team_code',
       'away_team_code', 'home_shots_off_goal', 'away_shots_off_goal',
       'home_chances', 'away_chances', 'home_shot_creation_ratio',
       'away_shot_creation_ratio', 'home_target_ratio', 'away_target_ratio',
       'home_conversion_rate', 'away_conversion_rate',
       'home_target_to_goal_ratio', 'away_target_to_goal_ratio',
       'home_goals_rolling_avg', 'home_conceded_goals_rolling_avg',
       'home_shots_rolling_avg', 'home_conceded_shots_rolling_avg',
       'home_shots_on_goal_rolling_avg',
       'home_c

In [4]:
df.shape

(3391, 66)

In [5]:
#use random forest regression to find the number of goals
selected_features = ['home_shots_rolling_avg','home_shots_on_goal_rolling_avg', 'home_target_to_goal_ratio_rolling_avg',
                     'home_conversion_rate_rolling_avg','home_shot_creation_ratio_rolling_avg', 'home_shots_off_goal_rolling_avg']

                   

In [7]:
X = df[selected_features]
y = df['home_goals'] 

cutoff_date = '2023-01-01'
df_past = df[df['date'] < cutoff_date]
df_future = df[df['date'] >= cutoff_date]
#split into training and testing data
X_train = df_past[selected_features]
y_train = df_past['home_goals']
X_test = df_future[selected_features]
y_test = df_future['home_goals']



In [233]:
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=45)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=45, ...)

In [207]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)
#Wprint('Length of our Training data:', X_train.shape, '\nLength of our Testing data:', y_test.shape)



Length of our Training data: (2712, 6) 
Length of our Testing data: (679,)


In [8]:
model = RandomForestRegressor(max_depth=2, n_estimators=100, random_state=45)


#fitting the model
model.fit(X_train, y_train)


RandomForestRegressor(max_depth=2, random_state=45)

In [210]:
# Get predictions at 95% prediction intervals and median.
df['home_conversion_rate'].head()


0    0.000000
1    0.000000
2    0.000000
3    0.148148
4    0.071429
Name: home_conversion_rate, dtype: float64

In [213]:
df['home_conversion_rate_rolling_avg'].head(30)

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.111111
11    0.041667
12    0.062500
13    0.062500
14    0.076923
15    0.111111
16    0.111111
17    0.000000
18    0.166667
19    0.176471
20    0.071429
21    0.147059
22    0.071429
23    0.199074
24    0.000000
25    0.179144
26    0.079412
27    0.033333
28    0.074176
29    0.102381
Name: home_conversion_rate_rolling_avg, dtype: float64

In [215]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 2,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 45,
 'verbose': 0,
 'warm_start': False}

In [9]:
y_pred = model.predict(X_test)

In [10]:
mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
r2 = r2_score(y_true=y_test, y_pred=y_pred)

print("Mean Squared Error (MSE): %f" % mse)
print("Mean Absolute Error (MAE): %f" % mae)
print("R-squared (R2 Score): %f" % r2) 

Mean Squared Error (MSE): 1.699407
Mean Absolute Error (MAE): 1.036580
R-squared (R2 Score): 0.070754


In [11]:
# Predicting goals scored for each sample in the test set
predictions = model.predict(X_test)

# Combine actual and predicted values for comparison
results_df = pd.DataFrame({
    'Actual Goals': y_test.values,
    'Predicted Left Goals': y_pred[:, 0],  
    'Predicted Median Goals': y_pred[:, 1]  # Use the median prediction
    
    ,'Predicted High Goals': y_pred[:, 2]  
}, index=y_test.index)

results_df['in_range'] = np.where((results_df['Actual Goals'] >= results_df['Predicted Left Goals']) & (results_df['Actual Goals'] <= results_df['Predicted High Goals']), 1, 0)



# Adding team information if available in the dataset
if 'home_team' in df.columns:
    results_df['Home Team'] = df.loc[y_test.index, 'home_team']

print("Average in_range pred: ", results_df['in_range'].mean())

# Display the comparison
print("\nPrediction Results:")
results_df.head(10)  # Display first 10 predictions

# Optional: Evaluate overall accuracy by computing absolute differences
# results_df['Error (Absolute)'] = abs(results_df['Actual Goals'] - results_df['Predicted Goals'])

# print("\nAverage Prediction Error (Absolute):", results_df['Error (Absolute)'].mean())


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
print('Score: ', r2_score(y_train, model.predict(X_train)))
# print('Score: ', r2_score(y_test, y_pred[:, 1]))
# print('MSE: ', mean_squared_error(y_test, y_pred[:, 1]))

Score:  0.1015781989316542


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed